# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

G. Mattia  ->  G. Mattia  |  ['G. Mattia']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
W. Brandner  ->  W. Brandner  |  ['W. Brandner']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
T. Henning  ->  T. Henning  |  ['T. Henning']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
S. Jiao  ->  S. Jiao  |  ['S. Jiao']
F. Xu  ->  F. Xu  |  ['F. Xu']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 83 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.19605


extracting tarball to tmp_2511.19605...

 done.


G. Mattia  ->  G. Mattia  |  ['G. Mattia']


...

 done.


/tmp/ipykernel_2848/2822249172.py:52: LatexWarning: 2511.20300 did not run properly
list index out of range
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2511.20386


extracting tarball to tmp_2511.20386...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.19605-b31b1b.svg)](https://arxiv.org/abs/2511.19605) | **Role of magnetic reconnection in blazar variability using numerical simulation**  |
|| C. K. Das, et al. -- incl., <mark>G. Mattia</mark> |
|*Appeared on*| *2025-11-26*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**|            Fast $\gamma$-ray variability in blazars remains a central puzzle in high-energy astrophysics, challenging standard shock acceleration models. Blazars, a subclass of active galactic nuclei (AGN) with jets pointed close to our line of sight, offer a unique view into jet dynamics. Blazar $\gamma$-ray light curves exhibit rapid, high-amplitude flares that point to promising alternative dissipation mechanisms such as magnetic reconnection. This study uses three-dimensional relativistic magnetohydrodynamic (RMHD) and resistive relativistic magnetohydrodynamic (ResRMHD) simulations with the PLUTO code to explore magnetic reconnection in turbulent, magnetized plasma columns. Focusing on current-driven kink instabilities, we identify the formation of current sheets due to magnetic reconnection, leading to plasmoid formation. We develop a novel technique combining hierarchical structure analysis and reconnection diagnostics to identify reconnecting current sheets. A statistical analysis of their geometry and orientation reveals a smaller subset that aligns closely with the jet axis, consistent with the jet-in-jet model. These structures can generate relativistically moving plasmoids with significant Doppler boosting, offering a plausible mechanism for the fast flares superimposed on slowly varying blazar light curves. These findings provide new insights into the plasma dynamics of relativistic jets and strengthen the case for magnetic reconnection as a key mechanism in blazar $\gamma$-ray variability.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.20005-b31b1b.svg)](https://arxiv.org/abs/2511.20005) | **Stellar Parameters of BOSS M dwarfs in SDSS-V DR19**  |
|| D. Qiu, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-11-26*|
|*Comments*| *19 pages, 12 Figures*|
|**Abstract**|            We utilized the Stellar LAbel Machine (SLAM), a data-driven model based on Support Vector Regression, to derive stellar parameters ([Fe/H], $T_{\rm eff}$, and $\log{g}$) for SDSS-V M dwarfs using low-resolution optical spectra (R$\sim$2000) obtained with the BOSS spectrographs. These parameters are calibrated using LAMOST F, G or K dwarf companions ([Fe/H]), and APOGEE Net ($T_{\rm eff}$ and $\log{g}$), respectively. Comparisons of SLAM predicted [Fe/H] values between two components of M+M dwarfs wide binaries show no bias but with a scatter of 0.11 dex. Further comparisons with two other works, which also calibrated the [Fe/H] of M dwarfs by using the F/G/K companions, reveal biases of -0.06$\pm$0.16 dex and 0.02$\pm$0.14 dex, respectively. The SLAM-derived effective temperatures agree well with the temperature which is calibrated by using interferometric angular diameters (bias: -27$\pm$92 K) and those of the LAMOST (bias: -34$\pm$65 K), but are systematically lower than those from an empirical relationship between the color index and $T_{\rm eff}$ by 146$\pm$45 K. The SLAM surface gravity aligns well with those of LAMOST (bias: -0.01$\pm$0.07 dex) and those derived from the stellar mass and radius (bias: -0.04$\pm$0.09 dex). Finally, we investigated a bias in [Fe/H] between SLAM and APOGEE ASPCAP. It depends on ASPCAP's [Fe/H] and $T_{\rm eff}$, we provide an equation to correct the ASPCAP metallicities.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.20091-b31b1b.svg)](https://arxiv.org/abs/2511.20091) | **Exomoon search with VLTI/GRAVITY around the substellar companion HD 206893 B**  |
|| Q. Kral, et al. -- incl., <mark>M. Benisty</mark>, <mark>W. Brandner</mark>, <mark>G. Chauvin</mark>, <mark>P. Garcia</mark>, <mark>T. Henning</mark>, <mark>L. Kreidberg</mark>, <mark>P. Mollière</mark> |
|*Appeared on*| *2025-11-26*|
|*Comments*| *accepted for publication in A&A*|
|**Abstract**|            Direct astrometric detection of exomoons remains unexplored. This study presents the first application of high-precision astrometry to search for exomoons around substellar companions. We investigate whether the orbital motion of the companion HD 206893 B exhibits astrometric residuals consistent with the gravitational influence of an exomoon or binary planet. Using the VLTI/GRAVITY instrument, we monitored the astrometric positions of HD 206893 B and c across both short (days to months) and long (yearly) timescales. This enabled us to isolate potential residual wobbles in the motion of component B attributable to an orbiting moon. Our analysis reveals tentative astrometric residuals in the HD 206893 B orbit. If interpreted as an exomoon signature, these residuals correspond to a candidate (HD 206893 B I) with an orbital period of approximately 0.76 years and a mass of $\sim$0.4 Jupiter masses. However, the origin of these residuals remains ambiguous and could be due to systematics. Complementing the astrometry, our analysis of GRAVITY $R=4000$ spectroscopy for HD 206893 B confirms a clear detection of water, but no CO is found using cross-correlation. We also find that AF Lep b, and $\beta$ Pic b are the best short-term candidates to look for moons with GRAVITY+. Our observations demonstrate the transformative potential of high-precision astrometry in the search for exomoons, and proves the feasibility of the technique to detect moons with masses lower than Jupiter and potentially down to less than Neptune in optimistic cases. Crucially, further high-precision astrometric observations with VLTI/GRAVITY are essential to verify the reality and nature of this signal and attempt this technique on a variety of planetary systems.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.19994-b31b1b.svg)](https://arxiv.org/abs/2511.19994) | **A Focused Review of Quintom Cosmology: From Quintom Dark Energy to Quintom Bounce**  |
|| T.-t. Qiu, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-11-26*|
|*Comments*| *19 pages, 11 figures*|
|**Abstract**|            The recently released data of DESI DR2 favors a dynamical dark energy theory, with the equation of state crossing the cosmological constant boundary $w=-1$. In this paper, we briefly review quintom cosmology, especially the quintom bounce. We will give three examples of a quintom bounce and one example of a cyclic universe with quintom matter.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.20242-b31b1b.svg)](https://arxiv.org/abs/2511.20242) | **Search for Radio Pulsations from Neutron Star Candidates in Detached Binaries**  |
|| S.-J. Gao, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2025-11-26*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**|            Recent optical astrometric and spectroscopic surveys have identified numerous neutron star (NS) candidates in non-accreting detached binary systems, but their compact-object nature remains unconfirmed. In this work, we present targeted radio observations of 31 such candidates using the Five-hundred-meter Aperture Spherical radio Telescope (FAST), the Robert C. Byrd Green Bank Telescope, and the Shanghai TianMa Radio Telescope. Over a total of 46.65 hours of observing time, we detected neither periodic nor single-pulse radio emissions. These nondetections place stringent upper limits on the flux densities of any potential radio signals, reaching ~4 $\mu$Jy for periodic emission and ~10 mJy for single pulses with FAST. Since our observations are highly sensitive and the flux density upper limits are well below the median fluxes of known Galactic pulsars, this suggests that geometric beaming is the most likely explanation for the non-detections if these objects are indeed pulsars. Alternatively, the NSs may be sufficiently old ($\gtrsim$ 10 Gyr) and have become intrinsically radio-quiet. In this case, our findings highlight the inherent difficulty of confirming NSs in such old detached binary systems through radio pulsation searches.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.20386-b31b1b.svg)](https://arxiv.org/abs/2511.20386) | **Performance Calibration of the Wavefront Sensor's EMCCD Detector for the Cool Planets Imaging Coronagraph Aboard CSST**  |
|| J. Dou, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-11-26*|
|*Comments*| *12pages, 11figures, 1table*|
|**Abstract**|            The wavefront sensor (WFS), equipped with an electron-multiplying charge-coupled device (EMCCD) detector, is a critical component of the Cool Planets Imaging Coronagraph (CPI-C) on the Chinese Space Station Telescope (CSST). Precise calibration of the WFS's EMCCD detector is essential to meet the stringent requirements for high-contrast exoplanet imaging. This study comprehensively characterizes key performance parameters of the detector to ensure its suitability for astronomical observations. Through a multi-stage screening protocol, we identified an EMCCD chip exhibiting high resolution and low noise. The electron-multiplying gain (EM Gain) of the EMCCD was analyzed to determine its impact on signal amplification and noise characteristics, identifying the optimal operational range. Additionally, noise properties such as readout noise were investigated. Experimental results demonstrate that the optimized detector meets CPI-C's initial application requirements, achieving high resolution and low noise. This study provides theoretical and experimental foundations for the use of EMCCD-based WFS in adaptive optics and astronomical observations, ensuring their reliability for advanced space-based imaging applications         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.20300-b31b1b.svg)](https://arxiv.org/abs/2511.20300) | **Tails of Gravity: Persistence of Star Formation in the CMZ Environment**  |
|| L. Feng, et al. -- incl., <mark>S. Jiao</mark>, <mark>F. Xu</mark> |
|*Appeared on*| *2025-11-26*|
|*Comments*| *25 pages, 17 figures, 2 tables, accepted for publication in ApJ*|
|**Abstract**|            We characterize star-forming gas in six molecular clouds (Sgr B1-off, Sgr B2, Sgr C, the 20 km s$^{-1}$ and 50 km s$^{-1}$ molecular clouds, and the Brick) in the Galactic central molecular zone (CMZ), and compare their star-forming activities with those in molecular clouds outside the CMZ. Using multi-band continuum observations taken from ${\it Planck}$, ${\it Herschel}$, JCMT/SCUBA-2, and CSO/SHARC2, we derived 8.5" resolution column density maps for the CMZ clouds and evaluated the column density probability distribution functions (N-PDFs). With the archival Atacama Large Millimeter/submillimeter Array (ALMA) 1.3 mm dust continuum data, we further evaluated the mass of the most massive cores ($M_{\rm core}^{\rm ma x}$). We find that the N-PDFs of four of the selected CMZ clouds are well described by a piecewise log-normal + power-law function, while the N-PDFs of the remaining two can be approximated by log-normal functions. In the first four targets, the masses in the power-law component ($M_{\rm gas}^{\rm bound}$), $M_{\rm core}^{\rm max}$, and star formation rate (SFR) are correlated. These correlations are very similar to those derived from low-mass clouds in the Solar neighborhood and massive star-forming regions on the Galactic disk. These findings lead to our key hypotheses: (1) In the extreme environment of the CMZ, the power-law component in the N-PDF also represents self-gravitationally bound gas structures, and (2) evolution and star-forming activities of self-gravitationally bound gas structures may be self-regulated, insensitive to the exterior environment on $\gtrsim$5-10 pc scales.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2511.19605/./Fig9.png', 'tmp_2511.19605/./Fig10.png', 'tmp_2511.19605/./Fig3.png']
copying  tmp_2511.19605/./Fig9.png to _build/html/
copying  tmp_2511.19605/./Fig10.png to _build/html/
copying  tmp_2511.19605/./Fig3.png to _build/html/
exported in  _build/html/2511.19605.md
    + _build/html/tmp_2511.19605/./Fig9.png
    + _build/html/tmp_2511.19605/./Fig10.png
    + _build/html/tmp_2511.19605/./Fig3.png
found figures []
exported in  _build/html/2511.20005.md
found figures ['tmp_2511.20091/./figures/hd206893b_exomoon_residuals_mod275.png', 'tmp_2511.20091/./figures/hd206893b_exomoon_residuals_phasecolor.png', 'tmp_2511.20091/./figures/HD206893B_spectrum_model_1400K.png', 'tmp_2511.20091/./figures/hd206893b_exomoon_corner_0.21-0.26au.png']
copying  tmp_2511.20091/./figures/hd206893b_exomoon_residuals_mod275.png to _build/html/
copying  tmp_2511.20091/./figures/hd206893b_exomoon_residuals_phasecolor.png to _build/html/
copying  tmp_2511.20091/./figures/HD206893B_spec

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ckd}[1]{\color{magenta}#1}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# Role of magnetic reconnection in blazar variability using numerical simulation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.19605-b31b1b.svg)](https://arxiv.org/abs/2511.19605)<mark>Appeared on: 2025-11-26</mark> -  _Accepted for publication in ApJ_

</div>
<div id="authors">

C. K. Das, et al. -- incl., <mark>G. Mattia</mark>

</div>
<div id="abstract">

**Abstract:** Fast $\gamma$ -ray variability in blazars remains a central puzzle in high-energy astrophysics, challenging standard shock acceleration models. Blazars, a subclass of active galactic nuclei (AGN) with jets pointed close to our line of sight, offer a unique view into jet dynamics. Blazar $\gamma$ -ray light curves exhibit rapid, high-amplitude flares that point to promising alternative dissipation mechanisms such as magnetic reconnection. This study uses three-dimensional relativistic magnetohydrodynamic (RMHD) and resistive relativistic magnetohydrodynamic (ResRMHD) simulations with the PLUTO code to explore magnetic reconnection in turbulent, magnetized plasma columns. Focusing on current-driven kink instabilities, we identify the formation of current sheets due to magnetic reconnection, leading to plasmoid formation. We develop a novel technique combining hierarchical structure analysis and reconnection diagnostics to identify reconnecting current sheets. A statistical analysis of their geometry and orientation reveals a smaller subset that aligns closely with the jet axis, consistent with the jet-in-jet model. These structures can generate relativistically moving plasmoids with significant Doppler boosting, offering a plausible mechanism for the fast flares superimposed on slowly varying blazar light curves. These findings provide new insights into the plasma dynamics of relativistic jets and strengthen the case for magnetic reconnection as a key mechanism in blazar $\gamma$ -ray variability.

</div>

<div id="div_fig1">

<img src="tmp_2511.19605/./Fig9.png" alt="Fig9" width="100%"/>

**Figure 9. -** Spatial, thickness and angular distribution of identified current sheets for two different simulation runs; _left:_**Res2g5p1** at $t = 234   t_0$ and _right:_**Res1g5p1** at $t = 246   t_0$. For both the case the column 1 represents XZ- and XY- slice plot of current density profile overplotted with identified current sheets ($S_{Ad}   \cap   S_{L}$) from _Astrodendro+LoRD_ method (represented in cyan colour). Top right: Thickness distribution of identified current sheets form both _Astrodendro_ and _Astrodendro+LoRD_ method. Bottom left: Angular distribution of identified current sheets moving at different angle with Z-axis. (*fig:ResCombDistg5*)

</div>
<div id="div_fig2">

<img src="tmp_2511.19605/./Fig10.png" alt="Fig10" width="100%"/>

**Figure 10. -** Spatial, thickness and angular distribution of identified current sheets for two different time steps for the simulation run; **Res0g10p1**_left:_ is at $t = 240   t_0$ and _right:_ is at $t = 258   t_0$. For both the case the column 1 represents XZ- and XY- slice plot of current density profile overplotted with identified current sheets ($S_{Ad}   \cap   S_{L}$) from _Astrodendro+LoRD_ method (represented in cyan colour). Top right: Thickness distribution of identified current sheets form both _Astrodendro_ and _Astrodendro+LoRD_ method. Bottom left: Angular distribution of identified current sheets moving at different angle with Z-axis. (*fig:NonResCombDistng10*)

</div>
<div id="div_fig3">

<img src="tmp_2511.19605/./Fig3.png" alt="Fig5" width="100%"/>

**Figure 5. -** Three-dimensional visualization of magnetic field lines (colored using a blue-hot color scale in the unit of $B_0$) overlaid with isosurfaces of fluid density (orange-red color scale in the unit of $\rho_0$) at four different time steps for the simulation run **Res0g5p1**. The snapshots correspond to: (a) $t = 123   t_0$, (b) $t = 153   t_0$, (c) $t = 183   t_0$, and (d) $t = 213   t_0$. These time steps capture the evolution of the magnetic field and the development of kink instability within the jet. (*fig:Res0g5p1_MF_evolution*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.19605"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{km\rm s^{-1}}$
$\newcommand{\masyr}{ mas \rm yr^{-1}}$
$\newcommand{\los}{\emph{los}}$
$\newcommand{\Usun}{{U_{\!\odot}}}$
$\newcommand{\Wsun}{{W_{\!\odot}}}$
$\newcommand{\Vsun}{{V_{\!\odot}}}$
$\newcommand{\Msun}{{M_{\odot}}}$
$\newcommand{\ikpc}{kpc^{-1}}$
$\newcommand{\teff}{T_{\mathrm{eff}}}$
$\newcommand{\logg}{\log{g}}$
$\newcommand{\feh}{[\mathrm{Fe/H}]}$
$\newcommand{\alp}{[\alpha/\mathrm{M}]}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\max}{{\rm max}}$
$\newcommand{\tot}{{\rm tot}}$
$\newcommand{\tidal}{{\rm tidal}}$
$\newcommand{\typ}{{\rm typ}}$</div>



<div id="title">

# Stellar Parameters of BOSS M dwarfs in SDSS-V DR19

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.20005-b31b1b.svg)](https://arxiv.org/abs/2511.20005)<mark>Appeared on: 2025-11-26</mark> -  _19 pages, 12 Figures_

</div>
<div id="authors">

D. Qiu, et al. -- incl., <mark>J. Li</mark>

</div>
<div id="abstract">

**Abstract:** We utilized the Stellar LAbel Machine (SLAM), a data-driven model based on Support Vector Regression, to derive stellar parameters ( $\feh$ , $\teff$ , and $\logg$ ) for SDSS-V M dwarfs using low-resolution optical spectra (R $\sim$ 2000) obtained with the BOSS spectrographs. These parameters are calibrated using LAMOST F, G or K dwarf companions ( $\feh$ ), and APOGEE Net ( $\teff$ and $\logg$ ), respectively.Comparisons of SLAM predicted $\feh$ values between two components of M+M dwarfs wide binaries show no bias but with a scatter of 0.11 dex. Further comparisons with two other works, which also calibrated the $\feh$ of M dwarfs by using the F/G/K companions, reveal biases of -0.06 $\pm$ 0.16 dexand 0.02 $\pm$ 0.14 dex, respectively. The SLAM-derived effective temperatures agree well with the temperature which is calibrated by using interferometric angular diameters (bias: -27 $\pm$ 92 K) and those of the LAMOST (bias: -34 $\pm$ 65 K), but are systematically lower than those from an empirical relationship between the color index and $\teff$ by 146 $\pm$ 45 K. The SLAM surface gravity aligns well with those of LAMOST (bias: -0.01 $\pm$ 0.07 dex) and those derived from the stellar mass and radius (bias: -0.04 $\pm$ 0.09 dex).  Finally, we investigated a bias in $\feh$ between SLAM and APOGEE ASPCAP. It depends on ASPCAP's $\feh$ and $\teff$ , we provide an equation to correct the ASPCAP metallicities.

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.20005"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\x}{\ensuremath{\mathbf{x}}}$
$\newcommand{\Rm}{\ensuremath{R_m}}$
$\newcommand{\Rr}{\ensuremath{R_c^\text{rad}}}$
$\newcommand{\Rt}{\ensuremath{R_c^\text{turb}}}$
$\newcommand{\pp}{\ensuremath{\mathbf{p}}}$
$\newcommand{\B}{\ensuremath{\mathbf{B}}}$
$\newcommand{\kk}{\ensuremath{\mathbf{k}}}$
$\newcommand{\ii}{\ensuremath{\text{i}}}$
$\newcommand{\w}{\ensuremath{\mathbf{w}}}$
$\newcommand{\ex}{\ensuremath{\mathbf{e}_{x}}}$
$\newcommand{\ey}{\ensuremath{\mathbf{e}_{y}}}$
$\newcommand{\ez}{\ensuremath{\mathbf{e}_{z}}}$
$\newcommand{\ephi}{\ensuremath{\mathbf{e}_{\phi}}}$
$\newcommand{\er}{\ensuremath{\mathbf{e}_{r}}}$
$\newcommand{\irchi}[2]{\raisebox{\depth}{#1\chi}}$
$\newcommand{\Re}{\ensuremath{R_e}}$
$\newcommand{\v}{\ensuremath{\mathbf{v}}}$
$\newcommand{\u}{\ensuremath{\mathbf{u}}}$
$\newcommand{\d}{\ensuremath{\partial}}$
$\newcommand\cesamxx{Cesam2k20\xspace}$
$\newcommand\eos{EoS\xspace}$
$\newcommand\erg{\mbox{erg}}$
$\newcommand\lunit{~\erg \mbox{s}^{-1} \mbox{cm}^{-2}}$
$\newcommand\K{\mbox{K}}$</div>



<div id="title">

# Exomoon search with VLTI/GRAVITY around the substellar companion HD 206893 B

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.20091-b31b1b.svg)](https://arxiv.org/abs/2511.20091)<mark>Appeared on: 2025-11-26</mark> -  _accepted for publication in A&A_

</div>
<div id="authors">

Q. Kral, et al. -- incl., <mark>M. Benisty</mark>, <mark>W. Brandner</mark>, <mark>G. Chauvin</mark>, <mark>P. Garcia</mark>, <mark>T. Henning</mark>, <mark>L. Kreidberg</mark>, <mark>P. Mollière</mark>

</div>
<div id="abstract">

**Abstract:** Direct astrometric detection of exomoons remains unexplored. This study presents the first application of high-precision astrometry to search for exomoons around substellar companions. We investigate whether the orbital motion of the companion HD 206893 B exhibits astrometric residuals consistent with the gravitational influence of an exomoon or binary planet. Using the VLTI/GRAVITY instrument, we monitored the astrometric positions of HD 206893 B and c across both short (days to months) and long (yearly) timescales. This enabled us to isolate potential residual wobbles in the motion of component B attributable to an orbiting moon. Our analysis reveals tentative astrometric residuals in the HD 206893 B orbit. If interpreted as an exomoon signature, these residuals correspond to a candidate (HD 206893 B I) with an orbital period of approximately 0.76 years and a mass of $\sim$ 0.4 Jupiter masses. However, the origin of these residuals remains ambiguous and could be due to systematics. Complementing the astrometry, our analysis of GRAVITY $R=4000$ spectroscopy for HD 206893 B confirms a clear detection of water, but no CO is found using cross-correlation. We also find that AF Lep b, and $\beta$ Pic b are the best short-term candidates to look for moons with GRAVITY+. Our observations demonstrate the transformativepotential of high-precision astrometry in the search for exomoons, and proves the feasibility of the technique to detect moons with masses lower than Jupiter and potentially down to less than Neptune in optimistic cases. Crucially, further high-precision astrometric observations with VLTI/GRAVITY are essential to verify the reality and nature of this signal and attempt this technique on a variety of planetary systems.

</div>

<div id="div_fig1">

<img src="tmp_2511.20091/./figures/hd206893b_exomoon_residuals_mod275.png" alt="Fig7.1" width="50%"/><img src="tmp_2511.20091/./figures/hd206893b_exomoon_residuals_phasecolor.png" alt="Fig7.2" width="50%"/>

**Figure 7. -**  Residuals of HD 206893 B orbit after removing the average motion due to HD 206893 B orbiting the star and accounting for the motion induced by the planet c. In the top two plots, the data are phase-folded by a period of 275 days, using MJD 58500 as the starting point. 50 random draws with SMA between 0.21 and 0.26 AU are plotted in cyan for the RA and orange for the Dec Vs time plots, respectively. These model draws have the average motion of HD 206893 B removed, so they show both perturbation from the exomoon model and the residuals in the orbit of B from the average orbit of B.  In the bottom plots, the data are plotted as a function of MJD and not phase-folded. The orbits are colored by orbital phase, where blue is an orbital phase of 0 and green is an orbital phase of 1. Note that the orbital periods vary between each model drawn and are slightly different from an exact 275-day orbital period. (*figresidualsmoon*)

</div>
<div id="div_fig2">

<img src="tmp_2511.20091/./figures/HD206893B_spectrum_model_1400K.png" alt="Fig9" width="100%"/>

**Figure 9. -**  Comparison between the GRAVITY combined spectrum (gray) and the ExoREM synthetic spectrum for $T=1400$ K (light blue). Each species spectrum is shown with an offset with different colours – dark blue for water, pink for sodium, and red for CO. (*figmodel*)

</div>
<div id="div_fig3">

<img src="tmp_2511.20091/./figures/hd206893b_exomoon_corner_0.21-0.26au.png" alt="Fig12" width="100%"/>

**Figure 12. -** Fit of the astrometric data of HD 206893 B, looking for an exomoon when we limited the prior to be between 0.21 and 0.26 au. The MCMC analysis finds a tentative exomoon candidate of mass around 0.5 Jupiter masses. (*figmcmcmoonfixed*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.20091"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

135  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

17  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
